In [1]:
# 为VOC文件夹下的图片提取特征并建立索引
# 1.遍历图片
# 2.提取特征
# 3.建立索引
# 4.存储文件名与索引匹配关系
# 5.保存索引到磁盘

In [1]:
# 导入相关包
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

In [2]:
# 获取所有图片的路径
image_list =  glob.glob('../VOCdevkit/VOC2012/JPEGImages/*.jpg')

In [3]:
len(image_list)

17125

In [4]:
# 导入自定义特征提取类
from feature_extract import FeatureExtract
# 实例化
feat = FeatureExtract()
import os
from tqdm.notebook import tqdm

ModuleNotFoundError: No module named 'tensorflow.python.trackable'

In [5]:
# 遍历图片，获取特征
# 特征列表
feat_list = []
# 文件名列表
name_list = []
for img_file in tqdm(image_list) :
    # 提取特征
    feat_vec = feat.extractFeat(img_file)
    # 将特征加入列表
    feat_list.append(feat_vec)
    # 解析文件名
    name = img_file.split(os.sep)[-1].split('.')[0]
    # 将文件名加入列表
    name_list.append(name)

    # break 

NameError: name 'tqdm' is not defined

In [7]:
# 将文件名列表存入Numpy
name_list = np.array(name_list)
np.save('./VOCdevkit/name_list.npy',name_list)

In [9]:
# 将特征存入faiss索引
import faiss

In [10]:
# 创建索引
index = faiss.IndexFlatL2(512)

In [11]:
#feat_list转为numpy数组
feat_list = np.array(feat_list)
# 将特征添加到索引中
index.add(feat_list)

In [12]:
# 查看索引大小
index.ntotal

17125

In [13]:
# 索引文件存入磁盘
faiss.write_index(index, './VOCdevkit/voc.index')

In [14]:
# 简单测试一下
k = 4
D,I = index.search(feat_list[:4],k)

In [15]:
I

array([[    0, 15554, 17028,  1419],
       [    1, 11630,  4966,  1156],
       [    2, 12515, 12409,  3271],
       [    3,  3118, 12603,  8689]], dtype=int64)